# BankChurn Predictor — Demo
Quick run: train small sample, predict, and show simple feature importance.

In [ ]:
import json
import pandas as pd
from pathlib import Path
from main import BankChurnPredictor
print('Imports ready')

In [ ]:
# Load data (expects data/raw/Churn.csv)
csv_path = Path('data/raw/Churn.csv')
if not csv_path.exists():
    # create small synthetic sample if not present
    import numpy as np
    n=500
    df = pd.DataFrame({
        'RowNumber': range(1,n+1),
        'CustomerId': range(10000,10000+n),
        'Surname': [f'Customer_{i}' for i in range(n)],
        'CreditScore': np.random.randint(350,851,n),
        'Geography': np.random.choice(['France','Spain','Germany'], n),
        'Gender': np.random.choice(['Male','Female'], n),
        'Age': np.random.randint(18,93,n),
        'Tenure': np.random.randint(0,11,n),
        'Balance': np.random.uniform(0,250000,n),
        'NumOfProducts': np.random.randint(1,5,n),
        'HasCrCard': np.random.choice([0,1], n),
        'IsActiveMember': np.random.choice([0,1], n),
        'EstimatedSalary': np.random.uniform(11,200000,n),
        'Exited': np.random.choice([0,1], n, p=[0.8,0.2])
    })
else:
    df = pd.read_csv(csv_path)
df.head()

In [ ]:
predictor = BankChurnPredictor('configs/config.yaml')
X, y = predictor.preprocess_data(df)
predictor.train(X.head(200), y.head(200))
from sklearn.model_selection import train_test_split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
preds, probs = predictor.predict(Xte)
print(json.dumps({'f1_demo': float((preds==yte).mean())}, indent=2))

In [ ]:
# Simple feature importance snapshot (using variance of columns as proxy if not available)
import numpy as np
imp = X.var().sort_values(ascending=False).head(10)
imp.plot(kind='bar', title='Feature importance (proxy)');